- packages

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests, schedule, time, psycopg2
from sqlalchemy import create_engine

- fucntions

In [2]:
def task():
    df = pd.DataFrame()
    urls = ["https://www.mercadolibre.com.mx/colchon-1-plaza-de-espuma-luuna-original-100cm-x-190cm-x-24cm/p/MLM15569488",
            'https://www.mercadolibre.com.mx/colchon-luuna-memory-foam-matrimonial-hecho-en-mexico/p/MLM15569489?product_trigger_id=MLM15569488&quantity=1',
            'https://www.mercadolibre.com.mx/colchon-luuna-memory-foam-queen-size-hecho-en-mexico/p/MLM12242066?product_trigger_id=MLM15569489&quantity=1',
            'https://www.mercadolibre.com.mx/colchon-king-de-espuma-luuna-original-190cm-x-200cm-x-24cm/p/MLM12242062?product_trigger_id=MLM15569488&quantity=1']

    for url in urls:
        prices = []
        sotcks = []
        response = requests.get(url)
        content = response.content

        soup = BeautifulSoup(content, "html.parser")

        price_elements = soup.find_all("span", class_="andes-money-amount__fraction")
        prices = [element.text for element in price_elements]    

        stock_elements = soup.find_all("p", class_="ui-pdp-color--BLACK ui-pdp-size--SMALL ui-pdp-family--SEMIBOLD ui-pdp-stock-information__title")
        stocks = [element.text for element in stock_elements]
        
        if len(prices) > 2:
            cu_price = int(str(prices[1]).replace(',', ''))
        elif len(prices) == 1:
            cu_price = 0
        else:
            None
        
        if len(stocks) > 0:
            cu_stock = str(stocks[0]).lower()
        else:
            cu_stock = 'stock no disponible'
        
        df_buffer = pd.DataFrame(data = {'price' : [cu_price], 'stock' : [cu_stock]})
        df = pd.concat([df, df_buffer], ignore_index = True)
        
        # credentials for the database
        usr = "****"
        pwd = "****"
        db =  "****"
        port = "****"

        # create the engine for the postgresql object
        engine = create_engine("postgresql://" + usr + ":" + pwd + "@localhost:5432/" + db)

        df.to_sql('zebrands', con = engine, schema = 'public', if_exists='append', index = False)

- main program

In [3]:
def mi_script():
    task()

schedule.every().day.at("08:00").do(mi_script)

while True:
    schedule.run_pending()
    time.sleep(1)

KeyboardInterrupt: 